# w205 Project 3 (Maria Auslander)

## Purpose of Project

The purpose of this project is to store and query game events using an API. This consists of the following tasks (taken from project statement):
* Send APU server log events to Kafka
* Create a data pipeline to capture events, using spark streaming to filter event types from Kafka and land them into HDFS and make them available for analysis using presto
* Use Apache Bench to create test data for the pipeline
* Create an analytics report which provides a description of the pipeline and an analysis of the events.

### Docker File

The docker file for this project is below:

```
---
version: '2'
services:
  zookeeper:
    image: confluentinc/cp-zookeeper:latest
    environment:
      ZOOKEEPER_CLIENT_PORT: 32181
      ZOOKEEPER_TICK_TIME: 2000
    expose:
      - "2181"
      - "2888"
      - "32181"
      - "3888"
    extra_hosts:
      - "moby:127.0.0.1"

  kafka:
    image: confluentinc/cp-kafka:latest
    depends_on:
      - zookeeper
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:32181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka:29092
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
    expose:
      - "9092"
      - "29092"
    extra_hosts:
      - "moby:127.0.0.1"

  cloudera:
    image: midsw205/hadoop:0.0.2
    hostname: cloudera
    expose:
      - "8020" # nn
      - "8888" # hue
      - "9083" # hive thrift
      - "10000" # hive jdbc
      - "50070" # nn http
    ports:
      - "8888"
    extra_hosts:
      - "moby:127.0.0.1"

  spark:
    image: midsw205/spark-python:0.0.6
    stdin_open: true
    tty: true
    volumes:
      - ~/w205:/w205
    expose:
      - "8888"
    ports:
      - "8889:8888" # 8888 conflicts with hue
    depends_on:
      - cloudera
    environment:
      HADOOP_NAMENODE: cloudera
      HIVE_THRIFTSERVER: cloudera:9083
    extra_hosts:
      - "moby:127.0.0.1"
    command: bash

  presto:
    image: midsw205/presto:0.0.1
    hostname: presto
    volumes:
      - ~/w205:/w205
    expose:
      - "8080"
    environment:
      HIVE_THRIFTSERVER: cloudera:9083
    extra_hosts:
      - "moby:127.0.0.1"

  mids:
    image: midsw205/base:0.1.9
    stdin_open: true
    tty: true
    volumes:
      - ~/w205:/w205
    expose:
      - "5000"
    ports:
      - "5000:5000"
    extra_hosts:
      - "moby:127.0.0.1"
```

The process was started by spinning up the docker cluster

``` 
docker-compose up -d
``` 

### Running the API Through Flask

Afterwards, I ran the app through flask:

```
docker-compose exec mids \
  env FLASK_APP=/w205/project-3-mariaauslander/game_api.py \
  flask run --host 0.0.0.0
```

The code for the game_api is listed below:

```
#!/usr/bin/env python
import json
from kafka import KafkaProducer
from flask import Flask, request
import random

app = Flask(__name__)
producer = KafkaProducer(bootstrap_servers='kafka:29092')


def log_to_kafka(topic, event):
    event.update(request.headers)
    producer.send(topic, json.dumps(event).encode())


@app.route("/")
def default_response():
    default_event = {'event_type': 'default'}
    log_to_kafka('events', default_event)
    return "This is the default response!\n"


@app.route("/purchase_a_sword")
def purchase_a_sword():
    sword_name=random.choice(['Zulfiqar','Joyeuse','Masamune','Curved Saber of San Martin','Durendal','Seven Branched Sword','Legbiter','Excalibur'])
    sword_type={'Zulfiqar':'Double bladed sword','Joyeuse':'Medieval coronation sword','Masamune':'Japanese sword created by Masamune','Curved Saber of San Martin':'Curved saber',
            'Durendal':'Medievel one-handed straight sword','Seven Branched Sword':'Japanese ceremonial sword','Legbiter':'gaddhjat sword from Norway','Excalibur':'sword of King Arthur'}
    purchase_sword_event = {'event_type': 'purchase_sword','item_type':'sword','item_name':sword_name,'item_desc':sword_type[sword_name]}
    log_to_kafka('events', purchase_sword_event)
    return "Sword Purchased!\n"

@app.route("/join_guild")
def join_guild():
    guild_name=random.choice(['corpus naviculariorum','Freemen of the City','Basoche','Za','Masonry','Carpentry','Bakery'])
    guild_type={'corpus naviculariorum':'guild of long-distance shippers in Rome','Freemen of the City':'Medieval guild whose members had freedom from serfdom',
            'Basoche':'French guild of legal clerks','Za':'Trade guild of Japan','Masonry':'Masonry guild','Carpentry':'Organization of carpenters','Bakery':'Organization of bakers'}
    join_guild_event={'event_type':'join_guild','item_type':'guild','item_name':guild_name,'item_desc':guild_type[guild_name]}
    log_to_kafka('events',join_guild_event)
    return "You joined a guild!\n"

@app.route("/choose_nemesis")
def choose_nemesis():
    nemesis_name=random.choice(['Ursula','Darth Vader','Loki','Lord Voldemort','Norman Bates'])
    nemesis_type={'Ursula':'Tormentor of the little mermaid','Darth Vader':'villain and evil ruler of the Star Wars franchise',
            'Loki':'Agitator of the Marvel Universe','Lord Voldemort':'Evil mind of the Harry Potter world','Norman Bates':'Fictional serial killer, a total psycho'}
    choose_nemesis_event={'event_type':'choose_nemesis','item_type':'nemesis','item_name':nemesis_name,'item_desc':nemesis_type[nemesis_name]}
    log_to_kafka('events',choose_nemesis_event)
    return "You have a nemesis!\n"
    
```

### Setting Up Kafka Topic (Events)

In a separate terminal window, I set up to watch the kafka topic "events":
```
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning
```

### Setting Up Streaming Pipeline to Read and Sort Events

Afterwards, in separate terminal windows, I set up streaming for three different event types:

<u>Sword Purchases</u>
```
docker-compose exec spark spark-submit /w205/project-3-mariaauslander/sword_stream.py
```

<u>Guilds Joined</u>
```
docker-compose exec spark spark-submit /w205/project-3-mariaauslander/guild_stream.py
```

<u>Nemeses Chosen</u>
```
docker-compose exec spark spark-submit /w205/project-3-mariaauslander/nemesis_stream.py
```

"sword_stream.py" is below, the other codes are very similar:
```
#!/usr/bin/env python
"""Extract events from kafka and write them to hdfs
"""
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType


def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- item_type: string (nullable = true)
    |-- item_name: string (nullable = true)
    |-- item_desc: string (nullable = true)
    """
    
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("item_type", StringType(), True),
        StructField("item_name", StringType(), True),
        StructField("item_desc", StringType(), True),
    ])


@udf('boolean')
def is_sword_purchase(event_as_json):
    """udf for filtering events
    """
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False


def main():
    """main
    """
    spark = SparkSession \
        .builder \
        .appName("ExtractEventsJob") \
        .getOrCreate()

    raw_events = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .load()

    sword_purchases = raw_events \
        .filter(is_sword_purchase(raw_events.value.cast('string'))) \
        .select(raw_events.value.cast('string').alias('raw_event'),
                raw_events.timestamp.cast('string'),
                from_json(raw_events.value.cast('string'),
                          purchase_sword_event_schema()).alias('json')) \
        .select('raw_event', 'timestamp', 'json.*')

    sink = sword_purchases \
        .writeStream \
        .format("parquet") \
        .option("checkpointLocation", "/tmp/checkpoints_for_sword_purchases") \
        .option("path", "/tmp/sword_purchases") \
        .trigger(processingTime="120 seconds") \
        .start()

    sink.awaitTermination()


if __name__ == "__main__":
    main()
```

The code "sword_stream.py" reads filters events from kafka to isolate sword purchase events and then continuously writes them to a stream in the folder "tmp/sword_purchases" (HDFS/parquet) until the code is terminated, either by stopping the code run itself or taking down the docker container. The other codes look for nemesis and guild events.

After starting the code runs listed above, I made sure the folders wrote to hadoop:
```
docker-compose exec cloudera hadoop fs -ls /tmp/
```

```
Found 9 items
drwxrwxrwt   - root   supergroup          0 2020-04-10 02:16 /tmp/checkpoints_for_guilds_joined
drwxrwxrwt   - root   supergroup          0 2020-04-10 02:17 /tmp/checkpoints_for_nemeses_chosen
drwxrwxrwt   - root   supergroup          0 2020-04-10 02:16 /tmp/checkpoints_for_sword_purchases
drwxr-xr-x   - root   supergroup          0 2020-04-10 02:16 /tmp/guilds_joined
drwxrwxrwt   - mapred mapred              0 2016-04-06 02:26 /tmp/hadoop-yarn
drwx-wx-wx   - hive   supergroup          0 2020-04-10 02:10 /tmp/hive
drwxrwxrwt   - mapred hadoop              0 2016-04-06 02:28 /tmp/logs
drwxr-xr-x   - root   supergroup          0 2020-04-10 02:17 /tmp/nemeses_chosen
drwxr-xr-x   - root   supergroup          0 2020-04-10 02:16 /tmp/sword_purchases
```

### Creating Tables in Hive

Afterwards, I created the tables through hive
```
docker-compose exec cloudera hive
```

##### Sword Purchases Table
```
create external table if not exists default.sword_purchases (
    raw_event string,
    timestamp string,
    separator string,
    Host string,
    User_Agent string,
    event_type string,
    item_type string,
    item_name string,
    item_desc string
  )
  stored as parquet 
  location '/tmp/sword_purchases'
  tblproperties ("parquet.compress"="SNAPPY");
```

##### Nemeses Chosen Table
```
create external table if not exists default.nemeses_chosen (
    raw_event string,
    timestamp string,
    separator string,
    Host string,
    User_Agent string,
    event_type string,
    item_type string,
    item_name string,
    item_desc string
  )
  stored as parquet 
  location '/tmp/nemeses_chosen'
  tblproperties ("parquet.compress"="SNAPPY");
```

##### Guilds Joined Table
```
create external table if not exists default.guilds_joined (
    raw_event string,
    timestamp string,
    separator string,
    Host string,
    User_Agent string,
    event_type string,
    item_type string,
    item_name string,
    item_desc string
  )
  stored as parquet 
  location '/tmp/guilds_joined'
  tblproperties ("parquet.compress"="SNAPPY");
```

##### Running and Querying Presto
Once the streaming and tables were set up through hive, I queried them through presto

```
docker-compose exec presto presto --server presto:8080 --catalog hive --schema default
```

```
show tables;

      Table      
-----------------
 guilds_joined   
 nemeses_chosen  
 sword_purchases 
```

### Using Apache Bench to Generate Data

With the tables set up, I added data using Apache Bench

The following commands were run to generate events as part of the data pipeline

<u>Default:</u>
```
docker-compose exec mids \
  ab \
    -n 10 \
    -H "Host: user1.comcast.com" \
    http://localhost:5000/
```

<u>Sword:</u>
```
docker-compose exec mids \
  ab \
    -n 10 \
    -H "Host: user1.comcast.com" \
    http://localhost:5000/purchase_a_sword
```

<u>Guild:</u>
```
docker-compose exec mids \
  ab \
    -n 10 \
    -H "Host: user1.comcast.com" \
    http://localhost:5000/join_guild
```

<u>Nemesis:</u>
```
docker-compose exec mids \
  ab \
    -n 10 \
    -H "Host: user1.comcast.com" \
    http://localhost:5000/choose_nemesis
```

### Querying Presto

To make sure the tables populated with ten rows each based on the commands above, I queried the tables using presto (in the same terminal used for Presto previously):

```
presto:default> select count(*) from sword_purchases;
 _col0 
-------
    10 
(1 row)


presto:default> select count(*) from guilds_joined;
 _col0 
-------
    10 
(1 row)


presto:default> select count(*) from nemeses_chosen;
 _col0 
-------
    10 
(1 row)

```

Afterwards, I fed the stream more data using the commands below:
```
while true; do
  docker-compose exec mids \
    ab -n 10 -H "Host: user1.comcast.com" \
      http://localhost:5000/purchase_a_sword
  sleep 10
done
```

```
while true; do
  docker-compose exec mids \
    ab -n 10 -H "Host: user1.comcast.com" \
      http://localhost:5000/join_guild
  sleep 5
done
```

```
while true; do
  docker-compose exec mids \
    ab -n 10 -H "Host: user1.comcast.com" \
      http://localhost:5000/choose_nemesis
  sleep 5
done
```

While more data was being streamed into the tables, I ran some queries to analyze the data within the tables.

#### Sword Purchase Events

##### Count of Sword Purchase Events
```
presto:default> select item_type, count(*) from sword_purchases group by item_type;
 item_type | _col1 
-----------+-------
 sword     |   490 
(1 row)
```

##### Count of Different Sword Types with Descriptions
```
presto:default> select item_name,item_desc, count(*) from sword_purchases group by item_name,item_desc;
         item_name          |             item_desc              | _col2 
----------------------------+------------------------------------+-------
 Masamune                   | Japanese sword created by Masamune |    66 
 Excalibur                  | Sword of King Arthur               |    57 
 Seven Branched Sword       | Japanese ceremonial sword          |    68 
 Joyeuse                    | Medieval coronation sword          |    53 
 Legbiter                   | Gaddhjalt sword from Norway        |    74 
 Durendal                   | Medievel one-handed straight sword |    67 
 Curved Saber of San Martin | Curved saber                       |    50 
 Zulfiqar                   | Double bladed sword                |    55 
(8 rows)
 ```
 
##### Top Three Most Common Sword Types Purchased
```
presto:default> select item_name, count(*) from sword_purchases group by item_name order by count(*) desc limit 3;
      item_name       | _col1 
----------------------+-------
 Legbiter             |    74 
 Seven Branched Sword |    68 
 Durendal             |    67 
(3 rows)
```

#### Guild Events

##### Count of Guild Events
```
presto:default> select item_type, count(*) from guilds_joined group by item_type;
 item_type | _col1 
-----------+-------
 guild     |  1040 
(1 row)
```

##### Count of Different Guild Types with Descriptions
```
presto:default> select item_name,item_desc, count(*) from guilds_joined group by item_name,item_desc;
       item_name       |                       item_desc                       | _col2 
-----------------------+-------------------------------------------------------+-------
 Bakery                | Organization of bakers                                |   147 
 Corpus Naviculariorum | Guild of long-distance shippers in Rome               |   160 
 Basoche               | French guild of legal clerks                          |   168 
 Freemen of the City   | Medieval guild whose members had freedom from serfdom |   159 
 Za                    | Trade guild of Japan                                  |   140 
 Carpentry             | Organization of carpenters                            |   152 
 Masonry               | Masonry guild                                         |   144 
(7 rows)
 ```
 
 ##### Top Three Most Common Guilds Joined
```
presto:default> select item_name, count(*) from guilds_joined group by item_name order by count(*) desc limit 3;
       item_name       | _col1 
-----------------------+-------
 Basoche               |   178 
 Freemen of the City   |   168 
 Corpus Naviculariorum |   166 
(3 rows)
```

#### Nemesis Events

##### Count of Nemesis Events
```
presto:default> select item_type, count(*) from nemeses_chosen group by item_type;
 item_type | _col1 
-----------+-------
 nemesis   |  1160 
(1 row)
```

##### Count of Different Nemeses with Descriptions
```
presto:default> select item_name,item_desc, count(*) from nemeses_chosen group by item_name,item_desc;
   item_name    |                     item_desc                     | _col2 
----------------+---------------------------------------------------+-------
 Norman Bates   | Fictional serial killer, a total psycho           |   239 
 Darth Vader    | Villain and evil ruler of the Star Wars franchise |   216 
 Loki           | Agitator of the Marvel Universe                   |   257 
 Ursula         | Tormentor of the little mermaid                   |   244 
 Lord Voldemort | Evil mind of the Harry Potter world               |   244 
(5 rows)
 ```
 
 ##### Top Three Most Common Nemeses
```
presto:default> select item_name, count(*) from nemeses_chosen group by item_name order by count(*) desc limit 3;
   item_name    | _col1 
----------------+-------
 Loki           |   272 
 Lord Voldemort |   260 
 Ursula         |   256 
(3 rows)
```

After querying the tables, I took down the docker container:
```
docker-compose down
```